# Part I: Crawling wikipedia for political figures
---


In [1]:
import time
import requests
from collections import deque
import pandas as pd
import urllib.parse
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
API_URL = "https://fa.wikipedia.org/w/api.php"

PREFIX = "سیاستمداران اهل ایران"   # look for categories that begin with this

paramssss = {
    "action":     "query",
    "list":       "allcategories",
    "acprefix":   PREFIX,
    "aclimit":    "1000",
    "format":     "json"
}

resp = requests.get(API_URL, params=paramssss).json()
cats = [c["*"] for c in resp["query"]["allcategories"]]

print(f"Found {len(cats)} categories beginning with “{PREFIX}”:\n")
for c in cats:
    print("–", c)

Found 10 categories beginning with “سیاستمداران اهل ایران”:

– سیاستمداران اهل ایران
– سیاستمداران اهل ایران بر پایه استان
– سیاستمداران اهل ایران بر پایه تبار یا قومیت
– سیاستمداران اهل ایران بر پایه حزب
– سیاستمداران اهل ایران بر پایه دوره
– سیاستمداران اهل ایران بر پایه سده
– سیاستمداران اهل ایران بر پایه شهر
– سیاستمداران اهل ایران در دوره صفویه
– سیاستمداران اهل ایران در دوره قاجار
– سیاستمداران اهل ایران در دوره پهلوی


# Part II: Running BFS on wikipedia pages
-----

In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote

# Global visited set to track all visited URLs across function calls
visited = set()

def get_wikipedia_pages(category_url, max_depth=3, current_depth=0):
    global visited
    
    if current_depth > max_depth or category_url in visited:
        return [], []
    
    visited.add(category_url)
    response = requests.get(category_url)
    if response.status_code != 200:
        print(f"Failed to retrieve page: {response.status_code}")
        return [], []
    
    soup = BeautifulSoup(response.text, "html.parser")
    pages = []
    subcategories = []

    # Extract pages
    for div in soup.find_all("div", class_="mw-category-group"):
        for li in div.find_all("li"):
            a_tag = li.find("a")
            if a_tag:
                title = a_tag.get_text()
                href = a_tag.get("href")
                if href and href.startswith("/wiki/"):  # Only pick valid article links
                    link = "https://fa.wikipedia.org" + href
                    link = unquote(link)  # decode %XX sequences into normal text
                    if "رده:" in link:
                        subcategories.append((title, link))
                    else:
                        pages.append((title, link))
    
    # Recursively process subcategories
    all_pages = pages.copy()
    all_subcategories = subcategories.copy()
    
    for _, subcat_link in subcategories:
        if subcat_link not in visited:
            sub_pages, sub_categories = get_wikipedia_pages(subcat_link, max_depth, current_depth + 1)
            all_pages.extend(sub_pages)
            all_subcategories.extend(sub_categories)
    
    return all_pages, all_subcategories

# Cherry-picked categories
seed_categories = [
    "سیاستمداران اهل ایران",
    "سیاستمداران اهل ایران بر پایه استان",
    "سیاستمداران اهل ایران بر پایه تبار یا قومیت",
    "سیاستمداران اهل ایران بر پایه حزب",
    "سیاستمداران اهل ایران بر پایه دوره",
    "سیاستمداران اهل ایران بر پایه سده",
    "سیاستمداران اهل ایران بر پایه شهر",
    "سیاستمداران اهل ایران در دوره صفویه",
    "سیاستمداران اهل ایران در دوره قاجار",
    "سیاستمداران اهل ایران در دوره پهلوی",
    "افراد بر پایه گرایش سیاسی اهل ایران",
    "افراد در دوره پهلوی",
    "فرماندهان ارتش شاهنشاهی ایران",
    "رده‌های با نام رئیس‌جمهورهای ایران",
    "رده‌های با نام نخست‌وزیران ایران"
]

pages = None
categories = None

for cat in seed_categories:
    category_url = f'https://fa.wikipedia.org/wiki/رده:{cat}'

    if pages is None:
        pages_, categories_ = get_wikipedia_pages(category_url, max_depth=10)
        pages = pages_
        categories = categories_
    else:
        pages_, categories_ = get_wikipedia_pages(category_url, max_depth=10)
        pages.extend(pages_)
        categories.extend(categories_)

    # Print results
print(f"Found {len(pages)} pages and {len(categories)} categories")
print("\nPages:")
for title, link in pages[:10]:  # Print first 10 pages
    print(f"{title}: {link}")

print("\nCategories:")
for title, link in categories[:10]:  # Print first 10 categories
    print(f"{title}: {link}")


Found 3505 pages and 185 categories

Pages:
بحث کاربر:007media: https://fa.wikipedia.org/wiki/بحث_کاربر:007media
بهمن آبادیان: https://fa.wikipedia.org/wiki/بهمن_آبادیان
کامبیز آتابای: https://fa.wikipedia.org/wiki/کامبیز_آتابای
عبدالقدیر آزاد: https://fa.wikipedia.org/wiki/عبدالقدیر_آزاد
حسام‌الدین آشنا: https://fa.wikipedia.org/wiki/حسام‌الدین_آشنا
فرج‌الله آصف: https://fa.wikipedia.org/wiki/فرج‌الله_آصف
مهرآفاق آصف‌زاده: https://fa.wikipedia.org/wiki/مهرآفاق_آصف‌زاده
کریم آصف: https://fa.wikipedia.org/wiki/کریم_آصف
علی آقامحمدی: https://fa.wikipedia.org/wiki/علی_آقامحمدی
یحیی آل اسحاق: https://fa.wikipedia.org/wiki/یحیی_آل_اسحاق

Categories:
تاریخ انتخاباتی سیاستمداران ایرانی: https://fa.wikipedia.org/wiki/رده:تاریخ_انتخاباتی_سیاستمداران_ایرانی
رایزنان فرهنگی ایران: https://fa.wikipedia.org/wiki/رده:رایزنان_فرهنگی_ایران
سران جنبش سبز: https://fa.wikipedia.org/wiki/رده:سران_جنبش_سبز
سیاستمداران ترورشده اهل ایران: https://fa.wikipedia.org/wiki/رده:سیاستمداران_ترورشده_اهل_ایران
سیاستمد

In [4]:
import pandas as pd

df2 = pd.DataFrame.from_records(pages).rename(columns={0: "title", 1: "link"}).set_index("title").drop_duplicates()
df2.to_csv("iran_political_figures.csv")

# Part III: Denoising the crawled data from non-person pages
---

In [6]:
# ── Load your CSV ──
csv_path = "iran_political_figures.csv"
df = pd.read_csv(csv_path, encoding="utf-8-sig")

# ── Prepare Wikidata API session ──
WIKIDATA_API = "https://www.wikidata.org/w/api.php"
session = requests.Session()
session.headers.update({
    "User-Agent": "IranPolCrawler/1.0 (youremail@example.com)"
})

def get_wd_entity_id(title: str) -> str | None:
    """
    Given a Persian Wikipedia title, return the corresponding Wikidata entity ID,
    or None if not found.
    """
    params = {
        "action": "wbgetentities",
        "sites": "fawiki",
        "titles": title,
        "format": "json"
    }
    r = session.get(WIKIDATA_API, params=params, timeout=10).json()
    entities = r.get("entities", {})
    for eid, data in entities.items():
        if eid != "-1":
            return eid
    return None

def is_human_entity(eid: str) -> bool:
    """
    Check if a given Wikidata entity ID has P31 (instance of) Q5 (human).
    """
    params = {
        "action": "wbgetclaims",
        "entity": eid,
        "property": "P31",
        "format": "json"
    }
    r = session.get(WIKIDATA_API, params=params, timeout=10).json()
    claims = r.get("claims", {}).get("P31", [])
    for claim in claims:
        dv = claim.get("mainsnak", {}).get("datavalue", {}).get("value", {})
        if dv.get("id") == "Q5":
            return True
    return False

# ── Run Wikidata-based human check ──
results = []
for title in tqdm(df["title"], desc="Wikidata human filter"):
    eid = get_wd_entity_id(title)
    if eid and is_human_entity(eid):
        results.append(True)
    else:
        results.append(False)
        # print(title)
    time.sleep(0.1)  # gentle throttle

df["is_person"] = results

# ── Split out persons vs. noise ──
df_person = df[df["is_person"]].drop(columns=["is_person"])
df_noise = df[~df["is_person"]].drop(columns=["is_person"])

# ── Print summary ──
print(f"Total entries:       {len(df)}")
print(f"Detected persons:    {len(df_person)}")
print(f"Filtered as noise:   {len(df_noise)}")

Wikidata human filter: 100%|██████████| 2513/2513 [35:10<00:00,  1.19it/s]

Total entries:       2513
Detected persons:    2344
Filtered as noise:   169


In [7]:
# Display a sample of the noise entries for review
print("Sample Noise Entries", df_noise.head(10))

df_person.to_csv("iran_political_figures_after_process.csv", index=False, encoding="utf-8-sig")
print("→ iran_political_figures_after_process.csv")

Sample Noise Entries                         title  \
0          بحث کاربر:007media   
1                بهمن آبادیان   
12               قربانعلی آهی   
15          ابراهیم فرحبخشیان   
16   ابراهیم‌خان یمین‌السلطنه   
37            میرطاهر اردبیلی   
44                اسحق فرهمند   
45           اسدالله جوانمردی   
60          اعظم‌الدوله زنگنه   
101   ویکی‌پدیا:ابوالقاسم خان   

                                                  link  
0     https://fa.wikipedia.org/wiki/بحث_کاربر:007media  
1           https://fa.wikipedia.org/wiki/بهمن_آبادیان  
12          https://fa.wikipedia.org/wiki/قربانعلی_آهی  
15     https://fa.wikipedia.org/wiki/ابراهیم_فرحبخشیان  
16   https://fa.wikipedia.org/wiki/ابراهیم‌خان_یمین...  
37       https://fa.wikipedia.org/wiki/میرطاهر_اردبیلی  
44           https://fa.wikipedia.org/wiki/اسحق_فرهمند  
45      https://fa.wikipedia.org/wiki/اسدالله_جوانمردی  
60     https://fa.wikipedia.org/wiki/اعظم‌الدوله_زنگنه  
101  https://fa.wikipedia.org/wiki/ویکی‌پدیا:

# Part III: Removing people with no infobox (indication of unreliable data)
---

In [10]:
# ── CONFIG ──
INPUT_CSV  = "iran_political_figures_after_process.csv"
OUT_WITH   = "with_infobox_politicians.csv"
OUT_NO     = "no_infobox_politicians.csv"
BASE_URL   = "https://fa.wikipedia.org/wiki/"

# ── Session with retries ──
session = requests.Session()
session.headers.update({
    "User-Agent": "IranPolCrawler/1.0 (youremail@example.com)",
    "Accept-Language": "fa"
})

def page_has_infobox(title):
    url = BASE_URL + urllib.parse.quote(title.replace(" ", "_"))
    try:
        resp = session.get(url, timeout=10)
        resp.raise_for_status()
    except requests.exceptions.RequestException as e:
        # network error or non-200 status
        print(f"[RequestError] {e} → {url}")
        return False

    soup = BeautifulSoup(resp.text, "html.parser")
    # first try the simple CSS selector
    tbl = soup.select_one("table.infobox")
    if tbl:
        return True

    # fallback: any <table> whose class attribute contains "infobox"
    tbl = soup.find("table", attrs={"class": lambda c: c and "infobox" in c})
    if tbl:
        return True

    return False

# ── Load titles ──
df = pd.read_csv(INPUT_CSV, encoding="utf-8-sig")

with_infobox = []
no_infobox   = []

for title in tqdm(df["title"], desc="Checking infobox"):
    has_box = page_has_infobox(title)
    entry  = {
        "name": title,
        "wikipedia_url": BASE_URL + urllib.parse.quote(title.replace(" ", "_"))
    }
    if has_box:
        with_infobox.append(entry)
    else:
        no_infobox.append(entry)

    time.sleep(0.2)  # be polite

# ── Save CSVs ──
pd.DataFrame(with_infobox).to_csv(OUT_WITH, index=False, encoding="utf-8-sig")
pd.DataFrame(no_infobox  ).to_csv(OUT_NO,   index=False, encoding="utf-8-sig")

print(f"✅ {len(with_infobox)} pages WITH infobox → {OUT_WITH}")
print(f"✅ {len(no_infobox)}  pages WITHOUT infobox → {OUT_NO}")

Checking infobox:   0%|          | 0/2344 [00:00<?, ?it/s]

Checking infobox: 100%|██████████| 2344/2344 [34:53<00:00,  1.12it/s]

✅ 2097 pages WITH infobox → with_infobox_politicians.csv
✅ 247  pages WITHOUT infobox → no_infobox_politicians.csv


In [11]:
print(pd.DataFrame(no_infobox).head(10))

                      name                                      wikipedia_url
0                 مجید آهی  https://fa.wikipedia.org/wiki/%D9%85%D8%AC%DB%...
1  ابوالفضل میرشمس شهشهانی  https://fa.wikipedia.org/wiki/%D8%A7%D8%A8%D9%...
2          مالک اژدر شریفی  https://fa.wikipedia.org/wiki/%D9%85%D8%A7%D9%...
3       میرزا حسین اصفهانی  https://fa.wikipedia.org/wiki/%D9%85%DB%8C%D8%...
4            امیر نجم رشتی  https://fa.wikipedia.org/wiki/%D8%A7%D9%85%DB%...
5        امیرحسین جهانشاهی  https://fa.wikipedia.org/wiki/%D8%A7%D9%85%DB%...
6            امیرقلی امینی  https://fa.wikipedia.org/wiki/%D8%A7%D9%85%DB%...
7                حسین اهری  https://fa.wikipedia.org/wiki/%D8%AD%D8%B3%DB%...
8     بهاءالدین محمد جوینی  https://fa.wikipedia.org/wiki/%D8%A8%D9%87%D8%...
9         حسن بن سهل سرخسی  https://fa.wikipedia.org/wiki/%D8%AD%D8%B3%D9%...
